# 4-SparkDataframe

This notebook demonstrates how to read and process a tabular datafile with the Spark(https://spark.apache.org/docs/latest/sql-programming-guide.html) library. Apache Spark is a unified analytics engine for large-scale parallel data processing. Spark can handle datasets that are large than the available memory (out-of-core) and process them in parallel on multiple cores.

Author: Peter W. Rose (pwrose@ucsd.edu)

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc
import time

In [2]:
#from jupyter_client import KernelManager
from jupyter_client import KernelManager
KernelManager.shutdown_wait_time = 60

If LOCAL_SCRATCH_DIR environment variable is not set, this notebook accesses the ../data directory for temporary files.

In [3]:
DATA_DIR = os.getenv("LOCAL_SCRATCH_DIR", default="../data")

### Setup Benchmark

The ```n_cores``` and ```file_format``` parameter are used for benchmarking ([see](7-ParallelEfficiency.ipynb)). 
This Cell [3] has been [parameterized](https://papermill.readthedocs.io/en/latest/usage-parameterize.html) as input parameters for [papermill](https://papermill.readthedocs.io/en/latest/index.html).

In [4]:
n_cores = 0 # default -> uses all available cores
file_format = "csv"
file_format = "parquet"

In [5]:
start = time.time()

### Setup Spark

In [6]:
if n_cores > 0:
    # use n_cores for benchmarking
    spark = SparkSession.builder.master(f"local[{n_cores}]").appName("SparkDataframe").getOrCreate()
else:
    # use all available cores
    spark = SparkSession.builder.appName("SparkDataframe").getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/29 16:13:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read Data

In [7]:
# read only specified columns
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id", "chromosome"]

if file_format == "csv":
    filename = os.path.join(DATA_DIR, "gene_info.tsv")
    genes = spark.read.option("header","true").option("sep", "\t").csv(filename)
elif file_format == "parquet":
    filename = os.path.join(DATA_DIR, "gene_info.parquet")
    genes = spark.read.parquet(filename)
else:
    print("invalid file format")
    
print("Filename:", filename)
file_size = os.path.getsize(filename)
print(f"File Size: {file_size/1E9:.1f} GB")

genes = genes.select(column_names)
genes = genes.withColumnRenamed("#tax_id", "tax_id")

Filename: ../data/gene_info.parquet
File Size: 0.0 GB


### Process Data

In [8]:
genes = genes.filter("type_of_gene == 'protein-coding'")
groups = genes.groupBy(["tax_id"]).count()
groups = groups.toDF("tax_id", "count")
groups = groups.sort(col("count").desc())

Convert Spark to Pandas dataframe (this triggers the computation)

In [9]:
groups = groups.toPandas()

In [10]:
spark.stop()

### Display Results

#### Number of human protein-coding genes (tax_id = 9606)

In [11]:
groups.query("tax_id == '9606'")

,tax_id,count
421,9606,20598


#### Top 5 organisms with the most protein-coding genes

In [12]:
groups.head()

,tax_id,count
0,4565,104035
1,3708,90975
2,90675,82686
3,94328,68154
4,3635,67632


In [13]:
end = time.time()

In [14]:
print(f"Spark: {end - start:.1f} sec.")

Spark: 23.0 sec.
